# Engineering Notebook: Design Exercise 3 Scale Models and Logical Clocks Simulation

### Start

In this project, we built a simulation of a small, asynchronous distributed system running on a single machine. The goal was to model multiple virtual machines that operate at different speeds and use Lamport logical clocks to maintain event ordering. Each machine:
- Picks a random clock rate between 1 and 6 ticks per second.
- Has its own message queue for incoming messages.
- Updates its Lamport clock based on local events, sends, and receives.
- Logs every event with details like system time, logical clock value, and current queue length.

This simulation helps us understand how causal ordering works in distributed systems, even when the machines run at different speeds.


## Implementation Details

### Virtual Machines (Processes)

- **Thread-Based Simulation:**  
  Each virtual machine is implemented as a separate Python thread. We found that using threads made it straightforward to simulate independent machines without dealing with the complexities of real network communication.

- **Clock Rate:**  
  Every machine randomly selects a tick rate (1–6 ticks per second). The tick duration (1 divided by the tick rate) controls how fast the machine runs its loop.


### Logical Clocks and Message Passing

- **Message Queues:**  
  We used Python’s queue.Queue to create an individual message queue for each machine. This approach effectively simulates asynchronous communication between the virtual machines.

- **Lamport Clock Rules:**  
  - **Internal Events:** The machine simply increments its logical clock by 1.  
  - **Send Events:** The machine increments its clock by 1, sends a message (containing the current clock value) to the target machine, and logs the event.  
  - **Receive Events:** When a message is received, the machine updates its clock to `max(local_clock, received_timestamp) + 1` and logs the receipt.


### Event Handling and Logging

- **Event Selection:**  
  During each tick, the machine first checks its message queue:
  - If a message is waiting, it processes that message.
  - If the queue is empty, it randomly decides whether to send a message (to one or both of the other machines) or to perform an internal event.

- **Logging:**  
  Every time an event occurs, the machine writes a log entry. Each log entry contains:
  - The event type (SEND, RECEIVE, INTERNAL)
  - The current system time (using `datetime.now()`)
  - The current logical clock value
  - Additional details (for example, which machine the message was sent to or where it came from)
  - The length of the message queue at that moment


### Stopping the Simulation

The simulation runs for a predetermined duration (for example here is 60 seconds). Once this time is up, a shared stop event is triggered, all machine threads wrap up their tasks, close their log files, and then the program exits.


## Design Decisions

- **Using Threads and Queues:**  
  We chose Python’s threads and in-memory queues because they are simple to work with and let us simulate a distributed system without the complexity of real network communication.

- **Implementing Lamport Clocks:**  
  The classic Lamport clock mechanism discussed in lecture was implemented since it is a fundamental concept in distributed systems. It helps us understand how to maintain event ordering without relying on physical clocks.

- **Random Event Selection:**  
  To mimic the unpredictable nature of distributed systems, we used randomness to decide if a machine will send a message or perform an internal event. This also allowed us to study how different event mixes affect the clock synchronization.

- **File-Based Logging:**  
  Each machine writes its events to a separate log file, which makes it easier to analyze the system’s behavior later by correlating the logs from different machines.


### Challenges and Considerations

1. **Simulating True Asynchrony:**  
   - **Challenge:** Real distributed systems run on separate machines. Simulating this on one machine means we need to carefully manage timing and communication between threads.
   - **Our Approach:** We used `time.sleep()` to simulate ticks and thread-safe queues to mimic network delays. However, the OS scheduling can sometimes introduce minor timing inaccuracies.

2. **Handling Clock Drift:**  
   - **Challenge:** Machines running at different tick rates will naturally have different logical clock values.
   - **Our Approach:** By using the update rule `max(local, received) + 1`, we ensure that clocks adjust correctly upon receiving messages. This lets us see how the clocks drift and then re-synchronize when communication occurs.

### Observations and Reflections

After running the simulation several times (multiple one-minute runs), we noted the following:
- **Logical Clock Jumps:**  
  When a machine gets a message with a higher timestamp, its clock jumps noticeably. This behavior shows the causal dependency between machines.

- **Message Queue Behavior:**  
  The size of the message queue varies over time. Sometimes, longer queues hint at periods of high communication traffic or slower processing.

- **Clock Drift:**  
  Machines running at slower tick rates sometimes diverge more in their logical clock values than faster machines, although regular message exchanges help bring them back in sync.

- **Event Mix Impact:**  
  A higher chance of sending messages helps keep the logical clocks more aligned across machines, while too many internal events can lead to larger differences.



## Implementation Details (Variant)

After our initial experiments with three virtual machines running with random clock rates between 1 and 6 ticks per second and a 70% chance of performing an internal event, we decided to explore another scenario. In this variant, we reduced the variation in clock cycles (using only 3 or 4 ticks per second) and lowered the probability of an internal event (down to 20%). This adjustment makes message passing more dominant in the simulation, and we wanted to see how this affects the synchronization and behavior of the Lamport clocks.

### Changes Made

1. **Smaller Variation in Clock Cycles:**  
   - **Before:** Each machine randomly picks a tick rate between 1 and 6 ticks per second, which determines how frequently it processes events. The tick duration is simply 1 divided by this tick rate.
   - **Now:** Each machine chooses from a much narrower range (either 3 or 4 ticks per second).  
   - **Why?**  
     This change reduces the timing differences between machines, which helps us observe whether the logical clocks become more aligned when the machines run at similar speeds.

2. **Lower Probability of Internal Events:**  
   - **Before:** A random integer from 1 to 10 was used, with internal events occurring 70% of the time (if the number was 4–10).  
   - **Now:** We use a random number between 0 and 1 with the following probabilities:
     - 30% chance to send a message to a random other machine.
     - 30% chance to send a message to the next machine in order.
     - 20% chance to send messages to both other machines.
     - 20% chance for an internal event.
   - **Why?**  
     By reducing the chance of internal events, we force more interactions between machines. This should help keep the logical clocks more synchronized and result in fewer large jumps due to message receptions.


### Observations and Reflections (Variant)

After running the variant simulation for several one-minute sessions, here’s what we observed:

- **Logical Clock Synchronization:**  
With the narrower tick rate (only 3 or 4 ticks per second) and more frequent message passing, the logical clocks stayed much closer together. The constant exchanges kept them from drifting apart.

- **Reduced Clock Jumps:**  
Because messages were sent more often, there were fewer instances of sudden, large jumps in the clock values. In the earlier setup, a message with a high timestamp could cause a big jump; now, the differences are more subtle.

- **Message Traffic:**  
The logs revealed that machines sent more messages and did fewer internal events. This made the overall system more interactive, reinforcing the causal ordering.

- **System Stability:**  
Overall, the system felt more stable, with less drift among the logical clocks—a useful observation for systems that need tight synchronization.


### Open ToDos
- Graph the logical clocks and queue sizes over time for each of the three machines, see if there is something interesting there

### Updates (Monday March 3)
- We have to use processes instead of threads with queues, so VBS changed the implementation to use processes
    - While at it, VBS also made the simulations a bit more configurable using the command line
    - The updated code is in ``simulate.py``

### Design Decisions
- Model each machine with the `Machine` class
    - Each machine gets an ID which is an integer between `0` and the total number of machines, in this case `3`
    - Each machine is then spun up as a separate process using Python's `concurrent.futures` module with its unique ID and is told a base port
    - Upon start up, each machine then sets up a listening socket at `base_port + id`. This way, each machine has its own unique port, and each machine knows how to contact the other machine (because they know their own port and the total number of machines)
    - Machines communicate using the sockets

- Event generation
    - Instead of explicitly generating numbers, we're using `random.choices` with the appropriate weights, which has the same effect; the weight of the interal event can be configured using the command line, the default is `7` (divide by `weight_internal + num_machines` to get probability of internal event, conditional on no messages in the queue)

- Tick speeds
    - You can configure the minimum and maximum tick count per second using the command line, with the options `--ticks-min` and `--ticks-max`, the default values are `1` and `6`, respectively

### Some initial observations
- As long as the values of `--ticks-min` and `--ticks-max` are relatively close (or more precisely, the realized tick speeds which we randomly generate based on these values), the jumps in logical clock times are relatively small and uniform, and the queue size stays low, mostly at `0`
- With tick speeds that are further apart (e.g. `1` and `20`), we see that in the slower machines, the jumps in logical clock times are pretty large, they have to overwhelmingly process receives, and their queue size becomes large after 60 seconds --- they can't keep up with all of the work that the faster machines are sending them and they can hardly ever send anything themselves

### Simulations
- We run each simulation for 60 seconds and repeat it 5 times
- We run three different simulations for tick speed: one where the ticks/second range uniformly from 1 to 6, one from 1 to 20, and one from 4 to 6.
- We run an additional simulation for less internal events, where the weight is reduced to 2 (out of 5 possible events), with ticks between 1 and 6

### Data Analysis

In [139]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

LOGS_FOLDER = Path.cwd() / "logs"
IMAGES_FOLDER = Path.cwd() / "images"

def eval(dir: Path, prefix: str, title: str):
    runs = []
    for run_folder in sorted(dir.glob(f"{prefix}*")):
        run_nr = int(run_folder.name.split("_run_")[1])
        for log_file in run_folder.glob("*.log"):
            log = pd.read_csv(log_file, sep="|", names=["machine_id", "timestamp_tmp", "event", "clock_tmp", "queue_length_tmp", "details"])
            log["run_nr"] = run_nr

            log["event"] = log["event"].str.strip()
            log["timestamp"] = pd.to_datetime(log["timestamp_tmp"])
            log["clock"] = log["clock_tmp"].str.lstrip("Clock: ").astype(int)
            log["queue_length"] = log["queue_length_tmp"].str.lstrip("Queue: ").astype(int)
            log["tick_speed"] = float(log.loc[0, "details"].split(",")[0].split(":")[1])
            log["runtime"] = float(log.loc[0, "details"].split(",")[1].split(":")[1])

            log = log.drop(["timestamp_tmp", "clock_tmp", "queue_length_tmp", "details"], axis=1)
            log = log.drop([log.index[0], log.index[-1]]) # remove start and stop events
            log = log.sort_values("timestamp")
            log = log.reset_index(drop=True)

            runs.append(log)

    df_complete = pd.concat(runs)

    max_run = df_complete["run_nr"].max()
    fig, ax = plt.subplots(max_run, 3, dpi=200, figsize=(3 * 5, max_run * 5))

    for run_nr, df in df_complete.groupby("run_nr"):
        # use relative time
        df["timestamp"] = (df["timestamp"] - df["timestamp"].min()).dt.total_seconds()
        
        # plot step function of logical clocks
        ax[0, 0].set_title("Logical Clocks Drift")
        for machine, m_df in df.groupby("machine_id"):
            ax[run_nr - 1, 0].step(m_df["timestamp"], m_df["clock"], label=f"{machine} ({m_df.at[0, 'tick_speed']:.2f}s/tick)")
            ax[run_nr - 1, 0].set_xlabel("Time elapsed [s]")
            ax[run_nr - 1, 0].set_ylabel("Logical clock")
            ax[run_nr - 1, 0].legend(loc="upper left")

        # plot step function of queue size
        ax[0, 1].set_title("Queue Length")
        for machine, m_df in df.groupby("machine_id"):
            ax[run_nr - 1, 1].step(m_df["timestamp"], m_df["queue_length"], label=f"{machine} ({m_df.at[0, 'tick_speed']:.2f}s/tick)")
            ax[run_nr - 1, 1].set_xlabel("Time elapsed [s]")
            ax[run_nr - 1, 1].set_ylabel("Queue length")
            ax[run_nr - 1, 1].legend(loc="upper left")

        # table with information
        table = []
        for machine, m_df in df.groupby("machine_id"):
            table.append([
                machine, # machine ID
                f'{m_df.at[0, "tick_speed"]:.3f}', # tick speed
                int(m_df["clock"].diff().max()), # max diff in clock
                int(m_df["queue_length"].max()), # max queue length
                f'{(m_df["event"] == "REC").mean():.1%}', # percentage of receive events
            ])

        ax[run_nr - 1, 2].axis("off")
        ax[run_nr - 1, 2].axis("tight")
        table = ax[run_nr - 1, 2].table(
            cellText=table, 
            colLabels=["ID", "s/tick", "Max\nClock\nJump", "Max\nQueue", "% REC"],
            loc="center",
        )

        table.auto_set_font_size(False)
        table.set_fontsize(12)
        table.scale(1, 5)  # Scale width and height of the cells

    fig.suptitle(title, y=0.91)

    return fig

In [140]:
fig = eval(LOGS_FOLDER, "ticks_1_20", "Ticks per second between 1 and 20")
fig.savefig(IMAGES_FOLDER / "plot_ticks_1_20.png", bbox_inches="tight")
plt.close(fig)

fig = eval(LOGS_FOLDER, "ticks_1_6", "Ticks per second between 1 and 6")
fig.savefig(IMAGES_FOLDER / "plot_ticks_1_6.png", bbox_inches="tight")
plt.close(fig)

fig = eval(LOGS_FOLDER, "ticks_4_6", "Ticks per second between 4 and 6")
fig.savefig(IMAGES_FOLDER / "plot_ticks_4_6.png", bbox_inches="tight")
plt.close(fig)

fig = eval(LOGS_FOLDER, "internal_2", "Ticks per second between 1 and 6, internal event weight of 2")
fig.savefig(IMAGES_FOLDER / "plot_internal_2.png", bbox_inches="tight")
plt.close(fig)

![Ticks between 1 and 6 per second](images/plot_ticks_1_6.png)
![Ticks between 1 and 20 per second](images/plot_ticks_1_20.png)
![Ticks between 4 and 6 per second](images/plot_ticks_4_6.png)
![Ticks between 1 and 6 per second, internal event weight of 2](images/plot_internal_2.png)

### Observations
- We see that for the fastest machine, the logical clock is *deterministically* a linear function over time, and of course the faster a machine is (absolutely speaking), the higher its logical clock goes within the 60 seconds that we run the simulation for
- Most notably, however, is that if one machine is much faster than another (say, 6 times as fast), the logical clocks diverge between the fastest and the slowest machine–this is because the slowest machine works the messages in a queue, i.e. it has already received messages further in the future while it's still processing older messages. We see this phenomenon also in the queue size: for the slower machine, it is unable to work through the queue and the queue sizes increases over time, whereas the fastest machine never has a queue length longer than 1 or 2. Another place this difference shows up is in the (unconditional) percentage of receive events: the slower the machine relative to the other machine it's communicating with, the higher the proportion of receive events; in some cases where the relative speed difference is especially large, this means that the slower machine has close to 100% receive events, while the faster machine has almost no receive events (because the slower machines never get around to sending any messages)
- The counterexample to this is the last plot, where we range the ticks between 4 and 6. The machines are roughly similar in speed, and so the queue sizes stay low, the logical clocks don't drift too far and end up synchronized, and the max clock jumps are relatively low, too.
- The max clock jumps are generally higher if the clocks eventually diverge/the machines have very different speeds but not much more so–even those where the speeds are pretty similar can have relatively high clock jumps of over 10.
- In the simulations with less frequent internal events but the same tick speeds between 1 and 6, we see that divergence is a bit more frequent and more extreme when it does occur. This makes sense as the fast machines send even more messages, while the slow receiving machine was only processing messages anyways, so now it just gets more messages on top of that
- The maximum clock jumps are also reduced in the simulations with fewer internal events, which makes sense given that relatively speaking more communication occurs, and so the clocks are synchronized more often